In [1]:
import cv2
#import mtcnn
from PIL import Image
from numpy import asarray,expand_dims
import numpy as np
import tensorflow as tf
from os import listdir
from os.path import isdir
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from tensorflow.keras.models import load_model 
import warnings
warnings.filterwarnings("ignore")

In [2]:
facenet = load_model('facenet_keras.h5')
facenet.load_weights('facenet_keras_weights.h5')

In [3]:
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [4]:
from joblib import dump,load
model = load('SVC_model.joblib')
encoder = load('OHE.joblib')
norm = load('Normalizer.joblib')

In [5]:
cvNet = cv2.dnn.readNetFromCaffe('architecture.txt','weights.caffemodel')
cap = cv2.VideoCapture(0)

img_size = 160
while True: 
    
    ret, frame = cap.read()
    
    (h, w) = frame.shape[:2]
    try:
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300,300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    except:
        pass
    cvNet.setInput(blob)
    detections = cvNet.forward()
    
    
    if detections != []:
        for i in range(0, detections.shape[2]):
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            image = frame[startY:endY, startX:endX]
            confidence = detections[0, 0, i, 2]
            
            if confidence > 0.99:
                try:
                    im = cv2.resize(image,(img_size,img_size))
                except:
                    pass
                im = np.array(im)/255.0
                im = im.reshape(160,160,3)
                face_array = asarray(im)
                
                embed = get_embedding(facenet,face_array)
            
                embed = expand_dims(embed,axis=0)
                embed = norm.transform(embed)
                yhat_class = model.predict(embed)
                yhat_prob = model.predict_proba(embed)
                predict_names = encoder.inverse_transform(yhat_class)
                
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                cv2.putText(frame, "Face : " + str(predict_names[0]), (startX, startY-50),
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
                cv2.putText(frame, "Prob : " + str(round(max(yhat_prob[0])*100)), (startX, startY-20),
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
    
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()